In [1]:
import copy

global k
global centers

In [2]:
# find the closest center and return its number in the list
def costCal(a,b):
    dimension = len(a)-1
    distance  = 0
    for i in range(dimension):
        distance = distance + (a[i]-b[i])**2
    return distance

def costClu(a,listB):
    costN = 0
    for z in range(len(listB)):
        costN = costN + costCal(a,listB[z])
    return costN

In [3]:
# find the closest center and return its number in the list
def findCluster(point):
    minDis = 66666 # should be a large number!!!
    cenP = -1
    for i in range(k):
        disTemp = costCal(centers[i],point)
        if disTemp < minDis:
            cenp = i
            minDis = disTemp
    #costTemp += minDis
    return cenp

In [4]:
# find the cluster which the point belongs to
def addCluster(point):
    return [point[0],point[1],point[2],findCluster(point)] #findCluster(point)

# update the cluster which the point belongs to
def newCluster(point):
    return [point[0],point[1],point[2],findCluster(point)]


In [ ]:
def createData(num):
    dataList = []
    for h in range(num):
        

In [5]:
# load data
path = "Skin_NonSkin.txt"
#sData = sc.textFile(path).map(lambda line: line.split("\t")).map(lambda arr:(int(arr[0]),int(arr[1]),int(arr[2]),0)) # append the list, label the cluster
sData = sc.textFile(path).map(lambda line: line.split("\t")).map(lambda arr:(int(arr[0]),int(arr[1]),int(arr[2]),int(arr[3])))
# test part
sData = sc.parallelize(sData.takeSample(False,2000,666))
print("cluster 1: " + str(sData.filter(lambda x: x[3]==1).count()))
print("cluster 2: " + str(sData.filter(lambda x: x[3]==2).count()))

print("Data count = " + str(sData.count()))


cluster 1: 422
cluster 2: 1578
Data count = 2000


In [6]:
# sample, k centers
k = 2
centers = sData.takeSample(False,k,666)
print(centers)

[(197, 196, 162, 2), (10, 9, 5, 2)]


In [7]:
# first iteration
upData = sData.map(lambda x: addCluster(x)) #find a cluster for each point

print("FirstRound count:" + str(upData.count()))

listTemp = upData.takeSample(False,k,666)
print(listTemp)

print("First Round...")
print("cluster 1:" + str(upData.filter(lambda x:x[3]==0).count()))
print("cluster 2:" + str(upData.filter(lambda x:x[3]==1).count()))

# calculate cost


FirstRound count:2000
[[197, 196, 162, 0], [10, 9, 5, 1]]
First Round...
cluster 1:1425
cluster 2:575


In [8]:
costSum  = 9999999999 # all clusters
costTemp = 9999999999 # converge or not

costCSum = 9999999999 # a cluster
costCTemp= 9999999999 # a cluster ,temporary

roundCnt = 0
while True:
    roundCnt+= 1
    costTemp = 0
    centerTemp = []
    for m in range(k):
        clu = upData.filter(lambda x:x[3]==m).collect()
        costCSum = 9999999999
        centerPos = -1
        for y in range(len(clu)):
            costCTemp = costClu(clu[y],clu) 
            if costCTemp < costCSum:
                costCSum = costCTemp
                centerPos = y
        costTemp   = costTemp + costCSum
        centerTemp.append(copy.deepcopy(clu[centerPos]))
                
    threshold = 1e-4
    if costTemp < (1-threshold) * costSum :
        centers = copy.deepcopy(centerTemp)
        costSum = costTemp
        #now the centers have been updated
        #re-classify
        upData = upData.map(lambda x: newCluster(x))
        print("cost" + str(costSum))
    else:
        print("Round:" + str(roundCnt))
        print("Result...")
        print(centers)
        print("cluster 1:" + str(upData.filter(lambda x:x[3]==0).count()))
        print("cluster 2:" + str(upData.filter(lambda x:x[3]==1).count()))
        break



cost12089644
cost11988461
cost11965225
Round:4
Result...
[[166, 165, 144, 0], [48, 64, 47, 1]]
cluster 1:1364
cluster 2:636
